In [ ]:
!pip install transformers
!pip install Rouge

In [ ]:
import pandas as pd
import nltk
import re
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import math
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/arabic_lyrics_df.csv')

In [ ]:
df.sample(10)

,title,lyrics
174069,من اول عمرى,ياحبيبي بحبك اد الدنيا بموت لو تبعد عنى فثانيه
261035,انت ايه,لو كان ده ذنبي ما اتوب عنه
234790,لما توعد حقو توفى,من غرورك ما بهمك مع المكتول منو
82986,عطشان والبحر جنبك,حرام تقضي العمر عطشان
216309,لأ مش انا,وعايزني ارجع تاني لا
269414,الشريف مبسوط مني,اللفظ اذهبيه ما علاقه شخصيه انا عرضى حفظاهو ما...
136070,عايز اعيش,واتكيفك زي احلى كيف
98509,فرق في حياتي,كفاية معاكي مبقتش وحداني
276199,قدام عينيك,أنا جي ثاني قدام عينيك بالشوق حبيبي بسال عليك
89898,ندمت عليك,ندمتا عليك ..... وندمتا عليك


In [ ]:
# Group the lyrics by 'songID' and join them
data = df.groupby('title')['lyrics'].apply('\n'.join).reset_index()
data = data.drop_duplicates(keep="first")
data

,title,lyrics
0,حالة غريبة,مقطع صوتى مقتبس من فيلم بالألوان الطبيعية -\n....
1,#NAME?,#NAME?\nسر الغيب فى حجاب مستور وانا ياطبيب داي...
2,.mvt,دمتم خلط ، دمتم debo niggs\nNbanniw\nاي فساد خ...
3,100 Marra,١٠٠ مرة\n\n١٠٠ مرة قولتلي انك فعلاً بتحبني\nو...
4,100 Wesh | ١٠٠ وش,[مقدمة: دياب]\nأعزائي الغدارين\nأعزائي الغداري...
...,...,...
16771,ﺍﻟﻠﻴﻠﻪ ﺳﺎﺭ,ﺍﻟﻠﻴﻠﻪ ﺳﺎﺭ ﻳﺎﻋﺸﺎﻳﺎ ﻋﺪﻳﻠﻪ ﻟﻲ ﺷﻮﻳﻢ ﺿﺎﺭﺏ ﺍﻟﺠﺪﺍﻳﺎ ...
16772,ﺍﻣﺴﻜﻰ ﻋﻠﻴﻚ ﻋﻴﻮﻧﻚ,ﺍﻣﺴﻜﻰ ﻋﻠﻴﻚ ﻋﻴﻮﻧﻚ\nﺩﻳﻞ ﺍﻣﺴﻜﻴﻬﻦ ﻋﻴﻮﻧﻚ\nﺩﻳﻞ\nﺧﻠﻴﺘ...
16773,ﺍﻻﻣﺎﻥ,ﺍﻻﻣﺎﻥ\nﻳﺸﻐﻞ ﻗﻠﺒﻲ ﺣﺒﻚ ﻋﻠﻰ ﻣﺮ ﺍﻟﺰﻣﺎﻥ\nﻭﺳﻠﻄﺎﻧﻪ ﺍﺳ...
16774,ﺑلغ ﺍﻻﻗﻮﺍﻝ,ﺑﻠّﻎ ﺍﻻﻗﻮﺍﻝ ﻳﺎﻫﻼﻝ\nﺷﺒﻴﻬﻚ ﺟﺴﻤﻲ ﺍﻧﻘﻼ\nﺍﻧﺖ ﺷﺎﻳﻒ ﺍ...


In [ ]:
#Lining up all the lyrics to create corpus
Corpus =''
for listitem in data.lyrics:
    Corpus += listitem

Corpus = Corpus.lower() #converting all alphabets to lowecase
print("Number of unique characters:", len(set(Corpus)))


Number of unique characters: 397


In [ ]:
#To See all the unique characters present in the Corpus
print("The unique characters:",sorted(set(Corpus)))

The unique characters: ['\n', ' ', '!', '"', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x82', '\xa0', '¤', '©', '«', '´', '»', '×', 'à', 'á', 'â', 'ã', 'ç', 'è', 'é', 'ê', 'í', 'î', 'ï', 'ó', '÷', 'ú', 'û', 'ü', 'ā', 'ğ', 'ī', 'ı', 'œ', 'ş', 'ū', 'а', 'г', 'е', 'и', 'к', 'л', 'м', 'н', 'ц', 'ч', 'щ', 'ю', 'я', 'א', 'ב', 'ה', 'ו', 'ז', 'ט', 'י', 'ך', 'ל', 'ם', 'מ', 'ן', 'נ', 'ס', 'ף', 'פ', 'צ', 'ק', 'ר', 'ת', '،', '؛', '؟', 'ء', 'آ', 'أ', 'ؤ', 'إ', 'ئ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ـ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ى', 'ي', 'ً', 'ٌ', 'ٍ', 'َ', 'ُ', 'ِ', 'ّ', 'ْ', 'ٓ', 'ٔ', '٠', '١', '٢', '٣', '٤', '٥', '٦', '٧', '٨', '٩', '٪'

In [ ]:
def _remove_english(  text):
    return re.sub(r'[a-zA-Z]+', '', text)

def _remove_diacritics(  text):
    arabic_diacritics = re.compile("""
                                ّ    | # Tashdid
                                َ    | # Fatha
                                ً    | # Tanwin Fath
                                ُ    | # Damma
                                ٌ    | # Tanwin Damm
                                ِ    | # Kasra
                                ٍ    | # Tanwin Kasr
                                ْ    | # Sukun
                               ـ    | # Tatwil/Kashida
                            """, re.VERBOSE)
    return re.sub(arabic_diacritics, '', text)

def _normalize_chars(  text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub("ڤ", "ف", text)
    text = re.sub("چ", "ج", text)
    text = re.sub("ژ", "ز", text)
    text = re.sub("پ", "ب", text)
    return text

def _remove_special_chars(  text):
    pattern = re.compile(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\d# ]+')
    return re.sub(pattern, ' ', text)

def _remove_punctuation(  text):
    pun = r"([!\"#$%\'\*\+,\.:;\-<=·>?@\^_ـ`{\|}~—٪’،؟`୍“؛”ۚ【»؛\s+«–…‘])"
    return re.sub(pun, ' ', text)

def _remove_whitespaces(  text):
    return re.sub(r'\s+', ' ', text).strip()

def _remove_repeated_chars(  text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def _remove_emojis(text):
    return re.sub(r'[^\w\s#@/:%.,_-]', '', text)

In [ ]:
def preprocess(text):
    text =  _remove_emojis(text)
    text =  _remove_english(text)
    text =  _remove_diacritics(text)
    text =  _normalize_chars(text)
    text =  _remove_special_chars(text)
    text =  _remove_punctuation(text)
    text =  _remove_whitespaces(text)
    text =  _remove_repeated_chars(text)
    return text

In [ ]:
data['lyrics']=data['lyrics'].apply(preprocess)

In [ ]:
#To See all the unique characters present in the Corpus
new_Corpus =''
for listitem in data.lyrics:
    new_Corpus += listitem

print("The unique characters:",sorted(set(new_Corpus)))

The unique characters: [' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'ء', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي', '٠', '١', '٢', '٣', '٤', '٥', '٦', '٧', '٨', '٩', 'ٱ', 'ٹ', 'ڕ', 'ڨ', 'ک', 'ڭ', 'ڵ', 'ھ', 'ۆ', 'ی', 'ێ', 'ے', 'ە', '۰', '۱', 'ݣ']


In [ ]:
data.sample(10)

,title,lyrics
12182,كلمة ودايقتها,من كل كلامي الي انا قلته طلعت كلمه ودايقتها وخ...
10003,طيب عادى,طيب عادي طيب عادي ابعد وعيش في بعادي قول اللي ...
10285,عذراء رمت الفؤاد,عذراء رمت الفءاد رمت الفءاد مليحه عذراء بسهام ...
2753,اجمل ما شافت عيني,اجمل ما شافت عيني بحس وهيا بتكلمني عيونها الحل...
6720,تحت فيحات الخميلة,تحت فيحاء الخميله كنا والاطيار تغني ترقص الاطي...
10603,علي صوتك بالغنا,علي صوتك بالغنا لسه الاغاني ممكنه ولسه ياما يا...
3393,اعاهدك,ليالي لقانا جميله كانت كانت ياسلام افراح وسمر ...
13802,مش فارقه كتير,مش فارقه كتير في قربك زي في غيابك هتفرق ايه سن...
11505,قدل شيخي,البشيل فوق الدبر مابميل قدل شيخي اللي رباعتو م...
13339,مافهمتنيش,دلوقتي نادم علي الكان علي كان دلوقتي نادم جاي ...


In [ ]:
data.to_csv('preprocessed_ar_data.csv', index=False)